In [1]:
from os import path
from collections import deque

from ga4stpg.graph import UGraph
from ga4stpg.graph.disjointsets import DisjointSets
from ga4stpg.graph.priorityqueue import PriorityQueue
from ga4stpg.graph.reader import read_problem
from ga4stpg.graph.util import has_cycle, how_many_components
from ga4stpg.util import STEIN_B

from ga4stpg.tree.evaluation import EvaluateTreeGraph
from ga4stpg.tree.generate import GenerateBasedPrimRST

In [2]:
INDEX_SELECT = 4

datasets = path.join('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary')

filename = path.join(datasets, STEIN_B[INDEX_SELECT][0])

print(filename)

STPG = read_problem(filename)

..\..\ppgi-stpg-gpx\datasets\ORLibrary\steinb5.txt


In [3]:
from ga4stpg.edgeset import UEdge, EdgeSet

class Segment:

    def __init__(self):
        self.edges = EdgeSet()
        self.cost = 0
        self.portals = set()

    def __len__(self):
        return len(self.edges)

    def __str__(self):
        return f'Segment <{len(self.edges)}>'

    def __iter__(self):
        return iter(self.edges)

    @property
    def bounds(self):
        return frozenset(self.portals)
    
    def add(self, v, u):
        self.edges.add(UEdge(v, u))
    

In [4]:
from random import randrange

def f_weight(v, u):
    if STPG.graph.has_edge(v, u):
        return STPG.graph.weight(v, u)
    else:
        return 0

def find_segments(graph, common_nodes):
    visited = set()

    # start = None
    # index = randrange(0, len(common_nodes))
    # for i, nro in enumerate(common_nodes):
    #     if i == index:
    #         start = nro
    #         break 
    # assert start is not None
    # stack_outter = [start]
    stack_outter = list(common_nodes)
    result = list()

    def search(start, neighbor):
        segment = Segment()
        segment.portals.add(start)
        segment.add(start, neighbor)
        segment.cost += f_weight(start, neighbor)
        
        stack_inner = [neighbor]

        while stack_inner:
            u = stack_inner.pop()
            visited.add(u)
            if u not in common_nodes:
                counter = 0
                for w in graph.adjacent_to(u):
                    if w not in visited:
                        stack_inner.append(w)
                        segment.add(u, w)
                        segment.cost += f_weight(u, w)
                        counter += 1
                if counter == 0:
                    segment.portals.add(u)                    
            else:
                stack_outter.append(u)
                segment.portals.add(u)
        # end while
        return segment
        # end search

    while stack_outter:
        s = stack_outter.pop()
        
        visited.add(s)
        for v in graph.adjacent_to(s):
            if v not in visited:
                seg = search(s, v)
                result.append(seg)

    return result

In [5]:
def check_portals(portals, disjoint):
    '''Verifica se os vértices portais de um segmento
    se conectam à mesma partição de vértices comuns. 

    Faz essa verificação em tempo O(n)
    '''
    f_check = set()

    for p in portals:
        if p not in disjoint:
            return False
        k = disjoint.find(p)
        if k in f_check:
            return False
        f_check.add(k)

    return True

In [6]:
edges = [
    (1, 3),
    (1, 15),
    (15, 17),
    (3, 5),
    (5, 11),
    (11, 13),
    (5, 7),
    (7, 9),
    (7, 23),
]

red = UGraph()

for v, u in edges:
    red.add_edge(v, u)

In [7]:
edges = [
    (1, 17),
    (1, 14),
    (14, 18),
    (18, 5),
    (5, 23),
    (23, 13),
    (5, 9)
]

blue = UGraph()

for v, u in edges:
    blue.add_edge(v, u)

## Crossing two random tree generated by Prim Random Steneir Tree

In [8]:
generator = GenerateBasedPrimRST(STPG)

red  = generator()
blue = generator()

In [9]:
red.edges == blue.edges

False

In [10]:
has_cycle(red), how_many_components(red)

(False, 1)

In [11]:
has_cycle(blue), how_many_components(blue)

(False, 1)

In [12]:
child     = UGraph()
red_only  = UGraph()
blue_only = UGraph()

In [13]:
for v, u in red.gen_undirect_edges():
    if blue.has_edge(v, u):
        child.add_edge(v, u)
    else:
        red_only.add_edge(v, u)

In [14]:
for v, u in blue.gen_undirect_edges():
    if not red.has_edge(v, u):
        blue_only.add_edge(v, u)

In [15]:
has_cycle(red_only), how_many_components(red_only)

(False, 8)

In [16]:
has_cycle(blue_only), how_many_components(blue_only)

(False, 10)

In [17]:
has_cycle(child), how_many_components(child)

(False, 9)

In [18]:
# 
red_leaves  = { v for v in red.vertices if red.degree(v) == 1}
blue_leaves = { v for v in blue.vertices if blue.degree(v) == 1}

In [19]:
common_nodes_red = set(red_only.vertices) & set(blue.vertices)

common_nodes_red

{3,
 4,
 5,
 6,
 9,
 10,
 12,
 14,
 15,
 18,
 20,
 21,
 23,
 24,
 25,
 26,
 31,
 32,
 34,
 36,
 37,
 38,
 39,
 40,
 42,
 45,
 46,
 47,
 48,
 49}

In [20]:
common_nodes_blue = set(blue_only.vertices) & set(red.vertices)

common_nodes_blue

{1,
 2,
 3,
 4,
 5,
 6,
 8,
 9,
 12,
 14,
 15,
 16,
 18,
 19,
 20,
 24,
 25,
 26,
 28,
 30,
 32,
 34,
 36,
 37,
 39,
 40,
 42,
 43,
 44,
 45,
 48,
 49}

In [21]:
red_segments  = find_segments(red_only, common_nodes_red)
blue_segments = find_segments(blue_only, common_nodes_blue)

In [22]:
len(red_segments), len(blue_segments)

(22, 24)

In [23]:
queue = PriorityQueue()

for seg in red_segments:
    # print(seg, seg.cost)
    queue.push(seg.cost, seg)

for seg in blue_segments:
    # print(seg, seg.cost)
    queue.push(seg.cost, seg)

len(queue)

46

In [24]:
common_nodes = set(red.vertices) | set(blue.vertices)



In [25]:
dset = DisjointSets()

nodes = set(child.vertices)
nodes.update(common_nodes)

for v in nodes:
    dset.make_set(v)

for v, u in child.gen_undirect_edges():
    dset.union(v, u)

In [26]:
len(dset)

50

In [27]:
dset.get_disjoint_sets()

defaultdict(set,
            {1: {1, 3, 17, 21},
             2: {2, 22, 47},
             28: {4, 28},
             5: {5},
             42: {6, 11, 16, 27, 42, 44},
             18: {7, 10, 13, 18, 19, 36, 38, 46},
             8: {8, 45},
             9: {9},
             12: {12},
             14: {14},
             15: {15},
             20: {20},
             31: {23, 29, 31, 33, 43},
             24: {24},
             30: {25, 30},
             26: {26},
             32: {32},
             34: {34},
             35: {35, 49},
             37: {37},
             39: {39},
             40: {40},
             41: {41},
             48: {48},
             50: {50}})

In [28]:
while queue:
    seg = queue.pop()
    
    if check_portals(seg.portals, dset):
        # print('adding segment', seg)
        print(seg)
        for edge in seg:
            print(edge, end="  ")
            v, u = edge
            child.add_edge(v, u)
        
        portals = iter(seg.portals)
        # Update dset 
        print("")
        p_last = next(portals)
        print("Portals: ", p_last, end=" ")
        for p in portals:
            print(p, end=" ")
            dset.union(p_last, p)
            p_last = p
        print("\n\n")
    # else:
    #     print("####### >>> rejecting: ", seg)

Segment <1>
Edge <(4, 24)>  
Portals:  24 4 


Segment <1>
Edge <(46, 47)>  
Portals:  46 47 


Segment <1>
Edge <(3, 20)>  
Portals:  3 20 


Segment <1>
Edge <(3, 14)>  
Portals:  3 14 


Segment <1>
Edge <(20, 39)>  
Portals:  20 39 


Segment <1>
Edge <(25, 43)>  
Portals:  25 43 


Segment <1>
Edge <(30, 32)>  
Portals:  32 30 


Segment <1>
Edge <(6, 50)>  
Portals:  50 6 


Segment <1>
Edge <(26, 31)>  
Portals:  26 31 


Segment <1>
Edge <(20, 49)>  
Portals:  49 20 


Segment <1>
Edge <(5, 25)>  
Portals:  25 5 


Segment <1>
Edge <(5, 18)>  
Portals:  18 5 


Segment <1>
Edge <(42, 49)>  
Portals:  49 42 


Segment <1>
Edge <(5, 48)>  
Portals:  48 5 


Segment <1>
Edge <(1, 18)>  
Portals:  1 18 


Segment <1>
Edge <(37, 40)>  
Portals:  40 37 


Segment <1>
Edge <(3, 34)>  
Portals:  34 3 


Segment <1>
Edge <(38, 45)>  
Portals:  45 38 


Segment <1>
Edge <(9, 25)>  
Portals:  9 25 


Segment <1>
Edge <(28, 30)>  
Portals:  28 30 


Segment <1>
Edge <(8, 37)>  
Portals:  8

In [29]:
for seg in red_segments:
    parent = dict()
    for p in seg.portals:
        if p in dset:
            parent[p] = dset.find(p)

    print(parent)

{49: 31, 4: 31}
{24: 31, 4: 31}
{4: 31, 37: 31}
{40: 31, 37: 31}
{40: 31, 10: 31}
{10: 31, 23: 31}
{34: 31, 23: 31}
{48: 31, 26: 31}
{26: 31, 31: 31}
{46: 31, 47: 31}
{46: 31, 6: 31}
{12: 31, 6: 31}
{21: 31, 6: 31}
{12: 31, 15: 31}
{45: 31, 38: 31}
{9: 31, 42: 31}
{9: 31, 25: 31}
{3: 31, 39: 31}
{3: 31, 20: 31}
{3: 31, 14: 31}
{32: 31, 36: 31}
{18: 31, 5: 31}


In [30]:
for seg in blue_segments:
    parent = dict()
    for p in seg.portals:
        if p in dset:
            parent[p] = dset.find(p)
    print(parent)

{49: 31, 42: 31}
{49: 31, 20: 31}
{20: 31, 39: 31}
{42: 31, 45: 31}
{48: 31, 5: 31}
{25: 31, 5: 31}
{25: 31, 43: 31}
{25: 31, 14: 31}
{25: 31, 15: 31}
{24: 31, 14: 31}
{19: 31, 44: 31}
{40: 31, 1: 31}
{1: 31, 18: 31}
{8: 31, 37: 31}
{9: 31, 36: 31}
{34: 31, 3: 31}
{32: 31, 30: 31}
{28: 31, 30: 31}
{2: 31, 28: 31}
{16: 31, 2: 31}
{2: 31, 12: 31}
{26: 31, 6: 31}
{50: 31, 6: 31}
{41: 31, 4: 31}


In [31]:
has_cycle(child)

False

In [32]:
how_many_components(child)

1

In [33]:
for seg in blue_segments:
    print(seg.portals)
    for edge in seg:
        print(edge, end='  ')
    print("")

{49, 42}
Edge <(42, 49)>  
{49, 20}
Edge <(20, 49)>  
{20, 39}
Edge <(20, 39)>  
{42, 45}
Edge <(42, 45)>  
{48, 5}
Edge <(5, 48)>  
{25, 5}
Edge <(5, 25)>  
{25, 43}
Edge <(25, 43)>  
{25, 14}
Edge <(14, 25)>  
{25, 15}
Edge <(15, 25)>  
{24, 14}
Edge <(14, 24)>  
{19, 44}
Edge <(19, 44)>  
{40, 1}
Edge <(1, 40)>  
{1, 18}
Edge <(1, 18)>  
{8, 37}
Edge <(8, 37)>  
{9, 36}
Edge <(9, 36)>  
{34, 3}
Edge <(3, 34)>  
{32, 30}
Edge <(30, 32)>  
{28, 30}
Edge <(28, 30)>  
{2, 28}
Edge <(2, 28)>  
{16, 2}
Edge <(2, 16)>  
{2, 12}
Edge <(2, 12)>  
{26, 6}
Edge <(6, 26)>  
{50, 6}
Edge <(6, 50)>  
{41, 4}
Edge <(4, 41)>  


In [34]:
for seg in red_segments:
    print(seg.portals)
    for edge in seg:
        print(edge, end='  ')
    print("")

{49, 4}
Edge <(4, 49)>  
{24, 4}
Edge <(4, 24)>  
{4, 37}
Edge <(4, 37)>  
{40, 37}
Edge <(37, 40)>  
{40, 10}
Edge <(10, 40)>  
{10, 23}
Edge <(10, 23)>  
{34, 23}
Edge <(23, 34)>  
{48, 26}
Edge <(26, 48)>  
{26, 31}
Edge <(26, 31)>  
{46, 47}
Edge <(46, 47)>  
{46, 6}
Edge <(6, 46)>  
{12, 6}
Edge <(6, 12)>  
{21, 6}
Edge <(6, 21)>  
{12, 15}
Edge <(12, 15)>  
{45, 38}
Edge <(38, 45)>  
{9, 42}
Edge <(9, 42)>  
{9, 25}
Edge <(9, 25)>  
{3, 39}
Edge <(3, 39)>  
{3, 20}
Edge <(3, 20)>  
{3, 14}
Edge <(3, 14)>  
{32, 36}
Edge <(32, 36)>  
{18, 5}
Edge <(5, 18)>  
